extract phone seqs from MFA alignments and save in format correct for
- speechbrain ASR training
- fastpitch TTS training

# imports

In [16]:
import os
import tgt 
from tqdm import tqdm

In [17]:
# print hostname to make sure we are on correct node
import socket
print(socket.gethostname())

kinloch.inf.ed.ac.uk


# global vars 

In [18]:
ALIGNMENT_DIR = '/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/LJSpeech-1.1/MFA_alignments_lowercase_nopunc'

# load alignments

In [19]:
# sanity check - assert that each utt has a corresponding alignment
alignment_files = set(os.listdir(ALIGNMENT_DIR))

In [21]:
uttid2textgrid = {}
for textgrid in tqdm(alignment_files):
    utt_id = textgrid.split('.')[0]
    tg_path = os.path.join(ALIGNMENT_DIR, textgrid)
    tg_obj = tgt.io.read_textgrid(tg_path, include_empty_intervals=True)
    uttid2textgrid[utt_id] = tg_obj

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13100/13100 [00:37<00:00, 349.97it/s]


# parse alignments to extract phone seq

In [22]:
utt2phones = {}
for utt_id, tg_obj in tqdm(uttid2textgrid.items()):
    phones = []
    for interval in tg_obj.get_tier_by_name('phones').intervals:
        phones.append(interval.text)
    utt2phones[utt_id] = phones

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13100/13100 [00:00<00:00, 42604.81it/s]


In [23]:
utt2phones

{'LJ032-0091': ['',
  'ð',
  'ɛ',
  'ɹ',
  'ɪ',
  'z',
  'n',
  'ow',
  '',
  'd',
  'ɑ',
  'k',
  't',
  'ɚ',
  'spn',
  '',
  'l',
  'aj',
  's',
  'ə',
  'n',
  's',
  't',
  't',
  'ə',
  'p',
  'ɹ',
  'æ',
  'k',
  't',
  'ɪ',
  's',
  'm',
  'ɛ',
  'd',
  'ɪ',
  's',
  'ɪ',
  'n',
  'ʔ',
  'ɪ',
  'n',
  'l',
  'ə',
  'w',
  'iː',
  'z',
  'i',
  'æ',
  'n',
  'ə',
  ''],
 'LJ041-0156': ['i',
  'p',
  'ɹ',
  'ɪ',
  'f',
  'ɝ',
  'd',
  'ð',
  'ə',
  '',
  'k',
  'w',
  'ow',
  'ʔ',
  'ɹ',
  'ɛ',
  'd',
  'ɑ',
  'ɹ',
  'mʲ',
  'i',
  '',
  'ɛ',
  'n',
  'd',
  'k',
  'w',
  'əw',
  't',
  ''],
 'LJ029-0120': ['w',
  'ɛː',
  'ð',
  'ə',
  'k',
  'ɹ',
  'aw',
  'd',
  'z',
  'w',
  'ʊ',
  'd',
  'bʲ',
  'iː',
  'ej',
  'b',
  'ɫ̩',
  't',
  'ə',
  'bʲ',
  'iː',
  'kʰ',
  'ə',
  'n',
  't',
  'ɹ',
  'əw',
  'ɫ',
  'd',
  'f',
  'ɒ',
  'ɹ',
  'ə',
  'ɟ',
  'ɹ',
  'ej',
  'ʔ',
  'd',
  'ɪ',
  's',
  't',
  'ɪ',
  'n',
  's',
  '',
  'ʔ',
  'æ',
  'n',
  'd',
  'w',
  'iː',
  'f',
  'ɪ',

# save in format for fastpitch

# save in format for speechbrain 

In [ ]:
def parse_textgrid(tier, speechreps, ignore_all_pauses=True):
    # latest MFA replaces silence phones with "" in output TextGrids
    total_num_frames = speechreps.size(0)
    sil_phones = ["sil", "sp", "spn", ""]
    utt_start_time = tier[0].start_time
    utt_end_time = tier[-1].end_time
    words = []
    word_durations = [] 
    sil_durations = []
    start_frames = []
    end_frames = []
    for i, t in enumerate(tier._objects):
        start, end, token = t.start_time, t.end_time, t.text
        start_idx, end_idx, num_frames = extract_reprs_with_timestamps(total_num_frames, start, end, utt_end_time)
    
        if token not in sil_phones:
            words.append(token)
            start_frames.append(start_idx)
            end_frames.append(end_idx)
            word_durations.append(num_frames)
        else:
            sil_durations.append(num_frames)

    return words, word_durations, sil_durations, start_frames, end_frames, utt_start_time, utt_end_time

words, word_durs, sil_durations, start_frames, end_frames, utt_start, utt_end = parse_textgrid(
    tg.get_tier_by_name('words'), reps, ignore_all_pauses=True
)